# Visualising ISS data starfish-decodings with napari

In [5]:
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from starfish import Experiment
from starfish.image import ApplyTransform, LearnTransform, Filter
from starfish.types import Axes
from starfish import data, FieldOfView
from starfish.spots import FindSpots
from starfish.util.plot import imshow_plane
from starfish.core.spots.DecodeSpots.trace_builders import build_spot_traces_exact_match


In [7]:
from starfish import Experiment
import napari

# Import ISS experiment:
e = Experiment.from_json('/Volumes/exchange/SebastianGonzalez/pag_2021/SpaceTx/primary/experiment.json')

e

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Let's visualise a single FOV

In [4]:
e.codebook

<xarray.Codebook (target: 9, r: 3, c: 3)>
array([[[1, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 1, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [1, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 1],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [1, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 1, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 1]]], dtype=uint8)
Coordinates:
  * target   (target) object 'Dppa4' 'Abca1' 'Lipg' ... 'Lars2' 'My166' 'Nr5a1'
  * r        (r) int64 0 1 2
  * c        (c) int64 0 1 2

In [ ]:
# The idea is for the user to select a field of view and then to
# show this sequence of files with the codebook information from
# the selected gene and plot the spots file on top of this

def view_gene_exp(
    gene: str = 'None', 
    fov: str = 'fov_000')